# Laboratorio Regresión Lineal


In [1]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Visualizaciones
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import tree

# Para realizar la regresión lineal y la evaluación del modelo
# -----------------------------------------------------------------------
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import sys
sys.path.append("../../")
from src.soporte_metricas import(
    generar_df_comparador,
    scatterplot_regresion,
    residual_plot
)

# Vamos a realizar la Regresión Lineal
Usando ambos metodos de Encoding utilizados

In [2]:
df_target = pd.read_csv("../../datos/03_Modelo3/06_autorenew_target_encoded.csv",index_col=0)
df_frequency = pd.read_csv("../../datos/03_Modelo3/06_autorenew_frequency_encoded.csv",index_col=0)

In [3]:
df_target.shape

(277391, 9)

# Que estoy haciendo?

Voy a ver las dos codificaciones, Frequency & Target, y veremos cual sale mejor, a partir de ahí vamos a intentar enseñar de forma visual que tan bien se adapta nuestro modelo a la realidad

# Frequency Encoder Train

In [4]:
Xfreq = df_frequency.drop(columns="price")
yfreq = df_frequency[["price"]]

### Dividimos los Grupos (70% train y 30% test)

In [5]:
Xfreq_train, Xfreq_test, yfreq_train, yfreq_test = train_test_split(Xfreq,yfreq,train_size=0.7,random_state=42)

### Generamos Regresión Lineal

In [6]:
modelofreq_lineal = LinearRegression()
modelofreq_lineal.fit(Xfreq_train,yfreq_train)

LinearRegression()

### Hacemos predicciones de Precio del 70% y el 30%
Con ello veremos que tan bien se le da a nuestro modelo predecir


In [7]:
yfreq_train_pred = modelofreq_lineal.predict(Xfreq_train)
yfreq_train_pred[:3]

array([[12818.78940546],
       [10606.63571907],
       [10386.16848607]])

In [8]:
yfreq_test_pred = modelofreq_lineal.predict(Xfreq_test)
yfreq_test_pred[:3]

array([[3343.67520089],
       [3898.68425333],
       [6447.82579313]])

In [9]:
metricas_freq = {
    'train': {
    'r2_score': r2_score(yfreq_train, yfreq_train_pred),
    'MAE': mean_absolute_error(yfreq_train, yfreq_train_pred),
    'MSE': mean_squared_error(yfreq_train, yfreq_train_pred),
    'RMSE': np.sqrt(mean_squared_error(yfreq_train, yfreq_train_pred))
    },
    'test': {
    'r2_score': r2_score(yfreq_test, yfreq_test_pred),
    'MAE': mean_absolute_error(yfreq_test, yfreq_test_pred),
    'MSE': mean_squared_error(yfreq_test, yfreq_test_pred),
    'RMSE': np.sqrt(mean_squared_error(yfreq_test, yfreq_test_pred))
    }
}

In [10]:
pd.set_option('display.float_format', '{:.2f}'.format)
pd.DataFrame(metricas_freq).T

,r2_score,MAE,MSE,RMSE
train,0.12,4762.79,142364671.49,11931.67
test,0.06,4955.57,383511241.39,19583.44


### Que vemos

- Vemos un R2 muy bajo y un índice de error muy alto
- Veamos con el otro método

# Target Encoded Train

In [11]:
df_target.sample()

,offerType,vehicleType,fuelType,gearbox,notRepairedDamage,kilometer,price,yearOfRegistration,powerCV_robust_scaler
104365,6621.23,5641.66,5850.27,11060.42,3628.73,6490.56,650.00,12863.00,2746.87


# Creamos dos DFs
- X es todo menos price
- Y es solo price

In [12]:
Xtarget = df_target.drop(columns="price")
ytarget = df_target[["price"]]

# Dividimos los grupos en un 70%
- Donde el train size sera el 70%.
- El test size el 30% restante.

In [13]:
Xtarget_train, Xtarget_test, ytarget_train, ytarget_test = train_test_split(Xtarget,ytarget,train_size=0.7,random_state=42)

# Aprendamos

In [14]:
modelotarget_lineal = LinearRegression()
modelotarget_lineal.fit(Xtarget_train,ytarget_train)

LinearRegression()

In [15]:
ytarget_train_pred = modelotarget_lineal.predict(Xtarget_train)
ytarget_train_pred[:3]

array([[ 6725.80112732],
       [ 7865.07164142],
       [20236.65554343]])

In [16]:
ytarget_test_pred = modelotarget_lineal.predict(Xtarget_test)
ytarget_test_pred[:3]

array([[2634.26984303],
       [ 366.63805325],
       [8036.068326  ]])

In [17]:
metricas_target = {
    'train': {
    'r2_score': r2_score(ytarget_train, ytarget_train_pred),
    'MAE': mean_absolute_error(ytarget_train, ytarget_train_pred),
    'MSE': mean_squared_error(ytarget_train, ytarget_train_pred),
    'RMSE': np.sqrt(mean_squared_error(ytarget_train, ytarget_train_pred))
    },
    'test': {
    'r2_score': r2_score(ytarget_test, ytarget_test_pred),
    'MAE': mean_absolute_error(ytarget_test, ytarget_test_pred),
    'MSE': mean_squared_error(ytarget_test, ytarget_test_pred),
    'RMSE': np.sqrt(mean_squared_error(ytarget_test, ytarget_test_pred))
    }
}

In [18]:
pd.set_option('display.float_format', '{:.2f}'.format)
df_metricas = pd.DataFrame(metricas_target).T
df_metricas

,r2_score,MAE,MSE,RMSE
train,0.29,3399.94,115065548.16,10726.86
test,0.14,3574.35,352213022.81,18767.34


# No parece una regresión Lineal
- Por la naturaleza de los datos no parece una regresión lineal
- También es posible que yo me haya equivocado, algo que es totalmente normal
- Pero realmente he intentado ajustar los datos a que puede ser real y que no puede ser real, hay cosas que de ser un error,en los datos, es algo totalmente válido en el mundo real
- Como análisis extra, podríamos revisar que los CV estuvieran dentro de un rango por marcas (es decir que volkswagen solo tiene coches de mínimo 75CV y máximo 420CV por ejemplo)

- Por otro lado, esta claro que de ambos encodings, Target encoding parece el modelo mejor entrenado y con un r2 más alto
- Aunque si revisamos el RMSE podemos ver que los errores son bastantes elevados, equivocarte 11.000€ ya es un error elevado, pero 16.000€ aún más
- Estamos hablando de que un coche que valga 25.000€ puede realmente estar valiendo 14.000€ o 36.000€ considero que es un error muy elevado!

¿Porque puede estar ocurriendo esto?
- Viendo que nuestro r2 no es muy alto pero no es extremadamente bajo y viendo nuestro RMSE considero que:
    - Estamos categorizando de forma muy específica y tal vez eso con la cantidad de datos que tenemos no se ajuste correctamente
    - Estamos ante unos datos que no se pueden tratar de forma lineal, el mercado del motor es un mercado que no podemos considerar linea, hay muchas variables que pueden afectar al precio y no es algo que pueda ser lineal (tal y como tengo los datos planteados)

# Visualicemos
- Vamos a ver que tan bien se ajusta nuestro modelo a la realidad


In [19]:
df_comparador = generar_df_comparador(Xtarget_train,Xtarget_test,ytarget_train,ytarget_test,ytarget_train_pred,ytarget_test_pred,"price_pred")

In [20]:
df_comparador

,offerType,vehicleType,fuelType,gearbox,notRepairedDamage,kilometer,yearOfRegistration,powerCV_robust_scaler,price,price_pred
0,6621.23,5641.66,5850.27,5308.59,3628.73,4091.64,2173.39,11660.70,480.00,4843.16
1,6621.23,13766.70,9125.11,5308.59,2539.11,6490.56,12863.00,15878.81,18300.00,19104.28
2,6621.23,13551.25,9125.11,11060.42,3628.73,6490.56,5626.24,8272.41,9800.00,11217.88
3,6621.23,3242.35,5850.27,5308.59,8033.46,4091.64,5626.24,2410.38,1500.00,-253.07
4,6621.23,3242.35,9125.11,5308.59,8033.46,8726.04,5626.24,5328.11,3600.00,6436.50
...,...,...,...,...,...,...,...,...,...,...
277386,6621.23,5641.66,3932.56,5059.53,3628.73,17750.85,5626.24,2200.00,2200.00,7211.05
277387,6621.23,11550.26,5850.27,11060.42,8033.46,6490.56,5626.24,2746.87,1199.00,6001.10
277388,6621.23,6917.86,9125.11,5308.59,8033.46,4091.64,2173.39,4821.82,9200.00,3159.55
277389,6621.23,6227.77,9125.11,5308.59,3628.73,4091.64,5626.24,2318.11,3400.00,1215.75


In [22]:
df_comparador.to_csv("../../datos/03_Modelo3/07_autorenew_prediccion_regresion_lineal.csv")

# Pintamos el Scatter Plot con X e Y

In [23]:
scatterplot_regresion(df_comparador,"price","price_pred")

# Sobre la gráfica
- Vemos que no seguimos una regresión lineal, por lo que hacer predicciones sobre la misma no es correcto
- Y bueno, yo de esto no sabía pero el área de la regresión crece a medida que sube, esto es el intervalo de confianza, que nos indica una incertidumbre alta en las predicciones para valores más altos y soporta mi teoría

# Lineplot

Lo he probado y queda descartado por su total y absoluta ilegilibilidad 

# Pasamos al Residual Plot
Pero esto que es??¿?¿?¿?¿?¿
,Bueno déjame ilustrarte

### Residual Plot
Tal y como su nombre indica, nos muestra los residuales del modelo.
- El residual es la diferencia que hay entre precios reales y los predichos
$$
\text{residual} = \text{precio\_real} - \text{precio\_predicho}
$$
- En un mundo ideal (en el que no vivimos) los valores residuales deberían estar alrededor de 0 sin mostrar patrones de ningún tipo

Es nuestra mejor opción, vamos a plotearla, pero antes, voy a mostrar una Gráfica Residual de un buen modelo:

In [24]:
# Crear un dataset sintético simulado de un buen modelo
# Generamos datos donde el modelo tiene una predicción casi perfecta
np.random.seed(42)
n = 500
x = np.linspace(0, 100, n)
y_real = 3 * x + 10 + np.random.normal(0, 10, n)  # Datos reales con un poco de ruido
y_pred = 3 * x + 10  # Predicciones perfectas (modelo lineal ideal)

# Calcular los residuales
residuals = y_real - y_pred

# Crear un DataFrame para visualización
df_good_model = pd.DataFrame({"price": y_real, "residual": residuals})

# Crear el residual plot
plt.figure(figsize=(12, 6))
sns.residplot(x="price", y="residual", data=df_good_model, lowess=True, color="green")

# Personalizar el gráfico
plt.axhline(0, color='red', linestyle='--', label='Residual = 0')
plt.title("Residual Plot de un Buen Modelo")
plt.xlabel("Precio Real (€)")
plt.ylabel("Residuales (€)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Sobre esta gráfica:
### Puntos verdes
- Cada punto verde es el residual o lo que viene a ser el error de predicción entonces:
    - Si el punto está en la línea roja (residual = 0), el modelo acertó!
    - Si el punto está por encima de la línea roja, el modelo dio un precio menor del precio real, es decir, lo subestimó ,se conoce como **Residual Positivo**
    - Si el punto está por debajo de la línea roja, el modelo dio un precio mayor del real, es decir lo sobreestimó, se conoce como **Residual Negativo**

- Si los puntos están dispersos de forma aleatoria y sin patrones (como vemos arriba) significa que el modelo está funcionando bien y está sacando los valores con errores mínimos
- Si hubiera un patrón, es decir, una curva o tendencia, nos indicaría que el modelo no está funcionando bien y deberíamos plantear el uso de un modelo distinto.
### Línea Verde
- Es la tendencia Promedio de los residuales a lo largo del rango de valores reales
- Dicho de otra forma, muestra cómo cambian los errores del modelo a medida que varían los valores reales.
    - Si se desvía hacia **arriba**: Indica que el modelo tiende a **subestima**r los valores
    - Si se desvía hacia **abajo**: Indica que el modelo tiende a **sobreestimar** los valores
### Línea Roja
- Indica el Residual 0, es decir, donde el precio predicho sería exactamente igual al original

# Veamos nuestra Realidad

In [25]:
residual_plot(df_comparador,"price","price_pred")

# Sobre Nuestra Gráfica
- Vemos que no seguimos el residual 0 con la línea morada
- Los puntos están bastante alejados del residual 0
- Que la línea morada suba indica que los errores tienden a aumentar a medida que aumentan los precios reales

Todo esto nos confirma una vez más lo que dijimos antes.
La regresión lineal no nos vale para este conjunto de datos.

Aunque creo que aquí se puede observar muy bien el porque. Si miras un Residual plot perfecto y este... pues como que no está bien.